# Finding leaflets in a double planar bilayer

This notebook demonstrates the performance of the "orientation" leaflet-finding algorithms on an example double planar bilayer of coarse-grained DPPC and CHOL lipids. Other algorithms are provided only for the first and easiest example (the entire membrane) as they cannot handle the two inner leaflets.

The particle groups analysed are:

* `u`: the entire membrane
* `half`: half the membrane, randomly dispersed
* `half_chol`: the CHOL molecules in `half`
* `fifth`: 1/5th the membrane, randomly dispersed
* `fifth_chol`: the CHOL molecules in `fifth`

In [1]:
import MDAnalysis as mda
import numpy as np
import nglview as nv
from MDAnalysis.analysis.leaflet import LeafletFinder
from MDAnalysis.tests.datafiles import Martini_double_membrane

_ColormakerRegistry()

In [2]:
u = mda.Universe(Martini_double_membrane)
half = u.residues[::2].atoms
half_chol = half.select_atoms('resname CHOL')
fifth = u.residues[::5].atoms
fifth_chol = fifth.select_atoms('resname CHOL')

In [13]:
def show_leaflets(ag, select="name PO4 GL1 ROH", n_groups=4, **kwargs):
    """Get and show coloured leaflets."""
    u = ag.universe
    lf = LeafletFinder(ag, select=select, n_groups=n_groups, **kwargs)
    n_residues = ', '.join(list(map(str, lf.sizes)))
    print(f"Found {len(lf.leaflets)} leaflets: {n_residues} residues")
    u.add_TopologyAttr('tempfactors')
    for i, lfl in enumerate(lf.leaflets):
        lag = lfl.residues.atoms
        lag.tempfactors = np.zeros(len(lag)) + i
    view = nv.show_mdanalysis(lf.residues)
    view.clear_representations()
    view.add_representation('spacefill', color_scheme='bfactor')
    return view

def get_leaflets(ag, select="name PO4 GL1 ROH", **kwargs):
    LeafletFinder(ag, select=select, **kwargs)

## Entire membrane

### Graph

Graph cannot handle the inner leaflets being so close.

In [14]:
show_leaflets(u, method="graph", cutoff=12)

Found 7 leaflets: 808, 407, 1, 1, 401, 1, 1 residues


NGLWidget()

![graph, full](images/lf_db_graph_full.png)

Images embedded to save space.

### Spectral clustering

Spectral clustering breaks down immediately. I'll leave it out from here on.

In [15]:
show_leaflets(u, method="spectralclustering", cutoff=80)

Found 4 leaflets: 469, 408, 403, 340 residues


NGLWidget()

![sc, full](images/lf_db_sc_full.png)

### Orientation

Orientation works.

In [16]:
show_leaflets(u, method="orientation")

Found 4 leaflets: 408, 408, 402, 402 residues


NGLWidget()

![orient, full](images/lf_db_or_full.png)

## Half the residues

In [17]:
show_leaflets(half, method="orientation")

Found 4 leaflets: 203, 203, 202, 202 residues


NGLWidget()

![orient, half](images/lf_db_or_half.png)

## Half CHOL

Default cutoff won't do here.

In [18]:
show_leaflets(half_chol, method="orientation")

Found 4 leaflets: 33, 31, 15, 11 residues


NGLWidget()

![orient, half chol, small cutoff](images/lf_db_or_half_chol_1.png)

A decent guideline is half the length of the membrane.

In [19]:
show_leaflets(half_chol, method="orientation", cutoff=60, n_groups=4)

Found 4 leaflets: 23, 23, 22, 22 residues


NGLWidget()

![orient, half chol, large cutoff](images/lf_db_or_half_chol_2.png)

## Fifth

In [20]:
show_leaflets(fifth, method="orientation", cutoff=100, n_groups=4)

Found 4 leaflets: 82, 82, 80, 80 residues


NGLWidget()

![orient, fifth](images/lf_db_or_fifth.png)

## Fifth CHOL

In [21]:
show_leaflets(fifth_chol, method="orientation", cutoff=60, n_groups=4)

Found 4 leaflets: 10, 10, 8, 8 residues


NGLWidget()

![orient, fifth chol](images/lf_db_or_fifth_chol.png)

In [ ]:
%timeit get_leaflets(fifth_chol, method="orientation", cutoff=60)

🦉